# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nhl_correlation(): 
    # YOUR CODE HERE
    nhl_df=pd.read_csv("assets/nhl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    # cleaning data
    # use only data from 2018
    nhl_df = nhl_df[nhl_df['year']==2018]
    # remove division infos
    nhl_df = nhl_df.drop(nhl_df[nhl_df['GP'].str.contains(r'Division')].index).reset_index(drop=True)
    # remove astericks from data with regexp
    nhl_df['team'] = nhl_df['team'].replace({'\*':''}, regex=True)
    # keep only games played, wins, losses and team
    nhl_df.drop(nhl_df.iloc[:, 4:], inplace = True, axis = 1)
    # drop games played
    nhl_df.drop(columns='GP', inplace=True, axis=1)
    # delete foírst names from teams and just keep last name to do the mapping
    nhl_df['team'] = nhl_df['team'].replace({'[\w.]*\ ':''}, regex=True)
    # convert dtypes to numeric data
    for i_col in nhl_df:
        if i_col != 'team':
            nhl_df[i_col] = pd.to_numeric(nhl_df[i_col], errors='coerce')
    # compute w/l rate
    nhl_df['w_l_rate'] = nhl_df['W'] / (nhl_df['W'] + nhl_df['L'])

    # cleaning cities data
    #rename cities column
    cities.rename(columns= {
        'Population (2016 est.)[8]' : 'Population'
    }, inplace=True)
    # remove brackets from dataset
    cities['NHL'] = cities['NHL'].replace({'\[.*\]':''}, regex=True)
    # mark empty string as nan
    cities.replace('', np.nan, inplace=True)
    # mark "-" as nan ("-" does not work so use sign directly from df)
    cities.replace(cities['NHL'][14], np.nan, inplace=True)
    # convert population series to pd numeric
    cities['Population'] = pd.to_numeric(cities['Population'], errors='coerce')
    #drop columns
    cities.drop(columns=['NBA', 'NFL', 'MLB'], inplace=True)
    # drop nas
    cities.dropna(inplace=True, how='any')
    cities.reset_index(drop=True, inplace=True)
    print(f"Number of cities with NHL teams: {len(cities['NHL']) - cities['NHL'].isna().sum()}")
    # split team names by capital and explode to a bigger df and reindex 
    cities = cities.assign(team = [re.findall("(?<!\s)[A-Z](?:[a-z\s]|(?<=\s)[A-Z])*", i_str) for i_str in cities['NHL']]).explode('team').reset_index(drop=True)
    # drop also foírst names from team to do the mapping
    cities['team'] = cities['team'].replace({'[\w.]*\ ':''}, regex=True)  
    # drop old NHL column
    cities.drop(columns=['NHL'], inplace=True)
    # merge dfs
    df = pd.merge(cities, nhl_df, 'outer', on='team')
    
    population_by_region = df.groupby(['Metropolitan area']).mean()['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = df.groupby(['Metropolitan area']).mean()['w_l_rate'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [2]:
nhl_correlation()

Number of cities with NHL teams: 28


0.012486162921209909

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [3]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nba_correlation():
    # YOUR CODE HERE
    
    nba_df=pd.read_csv("assets/nba.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    # cleaning data
    # use only data from 2018
    nba_df = nba_df[nba_df['year']==2018]

    # remove division infos
    #nhl_df = nhl_df.drop(nhl_df[nhl_df['GP'].str.contains(r'Division')].index).reset_index(drop=True)
    # remove astericks from data with regexp
    nba_df['team'] = nba_df['team'].replace({'\*':''}, regex=True)
    nba_df['team'] = nba_df['team'].replace({'\(\d*\)':''}, regex=True)
    # keep only games played, wins, losses and team
    nba_df = nba_df[['team', 'W/L%']]
    # drop games played
    #nhl_df.drop(columns='GP', inplace=True, axis=1)
    # delete first names from teams and just keep last name to do the mapping
    nba_df['team'] = nba_df['team'].replace({'[\w.]*\ ':''}, regex=True)
    nba_df['team'] = nba_df['team'].replace({'\s':''}, regex=True)
    # convert dtypes to numeric data
    for i_col in nba_df:
        if i_col != 'team':
            nba_df[i_col] = pd.to_numeric(nba_df[i_col], errors='coerce')

    #rename cities column
    cities.rename(columns= {
        'Population (2016 est.)[8]' : 'Population'
    }, inplace=True)
    # remove brackets from dataset
    cities['NBA'] = cities['NBA'].replace({'\[.*\]':''}, regex=True)
    # mark empty string as nan
    cities.replace('', np.nan, inplace=True)
    # mark "-" as nan ("-" does not work so use sign directly from df)
    cities.replace(cities['NBA'][16], np.nan, inplace=True)
    # convert population series to pd numeric
    cities['Population'] = pd.to_numeric(cities['Population'], errors='coerce')
    #drop columns
    cities.drop(columns=['NHL', 'NFL', 'MLB'], inplace=True)
    # drop nas
    cities.dropna(inplace=True, how='any')
    cities.reset_index(drop=True, inplace=True)
    print(f"Number of cities with NBA teams: {len(cities['NBA']) - cities['NBA'].isna().sum()}")
    # split team names by capital and explode to a bigger df and reindex 
    cities = cities.assign(team = [re.findall("(?<!\s)[A-Z](?:[a-z\s]|(?<=\s)[A-Z])*|[0-9]+[a-z]+", i_str) for i_str in cities['NBA']]).explode('team').reset_index(drop=True)
    # drop also foírst names from team to do the mapping
    cities['team'] = cities['team'].replace({'[\w.]*\ ':''}, regex=True)  
    # drop old NHL column
    cities.drop(columns=['NBA'], inplace=True)
    # merge dfs
    df2 = pd.merge(cities, nba_df, 'outer', on='team')
    
    population_by_region = df2.groupby(['Metropolitan area']).mean()['Population'] # pass in metropolitan area population from cities
    win_loss_by_region =  df2.groupby(['Metropolitan area']).mean()['W/L%']

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [4]:
nba_correlation()

Number of cities with NBA teams: 28


-0.17636350642182935

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [5]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def mlb_correlation(): 
    # YOUR CODE HERE
    
    mlb_df=pd.read_csv("assets/mlb.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    # cleaning data
    # use only data from 2018
    mlb_df = mlb_df[mlb_df['year']==2018]
    # remove division infos
    # mlb_df = mlb_df.drop(mlb_df[mlb_df['GP'].str.contains(r'Division')].index).reset_index(drop=True)
    # remove astericks from data with regexp
    mlb_df['team'] = mlb_df['team'].replace({'\*':''}, regex=True)
    # keep only teams and win-loose rate
    mlb_df = mlb_df[['team', 'W-L%']]

    # Because there are red and white concat the names before sox in both dfs to do the mapping
    mlb_df['team'] = mlb_df['team'].str.replace('\ Sox','Sox')
    # delete first names from teams and just keep last name to do the mapping
    mlb_df['team'] = mlb_df['team'].replace({'[\w.]*\ ':''}, regex=True)
    #mlb_df['team'] = mlb_df['team'].replace({'\s':''}, regex=True)
    # convert dtypes to numeric data
    for i_col in mlb_df:
        if i_col != 'team':
            mlb_df[i_col] = pd.to_numeric(mlb_df[i_col], errors='coerce')

    # cleaning cities data
    #rename cities column
    cities.rename(columns= {
        'Population (2016 est.)[8]' : 'Population'
    }, inplace=True)
    # remove brackets from dataset
    cities['MLB'] = cities['MLB'].replace({'\[.*\]':''}, regex=True)
    # mark empty string as nan
    cities.replace('', np.nan, inplace=True)
    # mark "-" as nan ("-" does not work so use sign directly from df)
    cities.replace(cities['MLB'][24], np.nan, inplace=True)
    # convert population series to pd numeric
    cities['Population'] = pd.to_numeric(cities['Population'], errors='coerce')
    #drop columns
    cities.drop(columns=['NBA', 'NFL', 'NHL'], inplace=True)
    # drop nas
    cities.dropna(inplace=True, how='any')
    cities.reset_index(drop=True, inplace=True)
    print(f"Number of cities with MLB teams: {len(cities['MLB']) - cities['MLB'].isna().sum()}")
    # split team names by capital and explode to a bigger df and reindex 
    cities = cities.assign(team = [re.findall("(?<!\s)[A-Z](?:[a-z\s]|(?<=\s)[A-Z])*", i_str) for i_str in cities['MLB']]).explode('team').reset_index(drop=True)
    # Because there are red and white concat the names before sox in both dfs to do the mapping
    cities['team'] = cities['team'].str.replace('\ Sox','Sox')
    # drop also foírst names from team to do the mapping
    cities['team'] = cities['team'].replace({'[\w.]*\ ':''}, regex=True)  
    # drop old NHL column
    cities.drop(columns=['MLB'], inplace=True)
    # merge dfs
    df3 = pd.merge(cities, mlb_df, 'outer', on='team')

    
    population_by_region = df3.groupby(['Metropolitan area']).mean()['Population'] # pass in metropolitan area population from cities
    win_loss_by_region =  df3.groupby(['Metropolitan area']).mean()['W-L%']

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [6]:
mlb_correlation()

Number of cities with MLB teams: 26


<ipython-input-5-7ccad7b5d39a>:27: FutureWarning: The default value of regex will change from True to False in a future version.
  mlb_df['team'] = mlb_df['team'].str.replace('\ Sox','Sox')
<ipython-input-5-7ccad7b5d39a>:58: FutureWarning: The default value of regex will change from True to False in a future version.
  cities['team'] = cities['team'].str.replace('\ Sox','Sox')


0.15003737475409495

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [7]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nfl_correlation(): 
    # YOUR CODE HERE
    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    # use only data from 2018
    nfl_df = nfl_df[nfl_df['year']==2018]
    # drop NFC and AFC
    nfl_df = nfl_df.drop(nfl_df[nfl_df['MoV'].str.contains(r'AFC')].index).reset_index(drop=True)
    nfl_df = nfl_df.drop(nfl_df[nfl_df['MoV'].str.contains(r'NFC')].index).reset_index(drop=True)

    # remove division infos
    #nhl_df = nhl_df.drop(nhl_df[nhl_df['GP'].str.contains(r'Division')].index).reset_index(drop=True)
    # remove astericks from data with regexp
    nfl_df['team'] = nfl_df['team'].replace({'\*':''}, regex=True)
    nfl_df['team'] = nfl_df['team'].replace({'\(\d*\)':''}, regex=True)
    # keep only games played, wins, losses and team
    nfl_df = nfl_df[['team', 'W-L%']]
    # drop games played
    #nhl_df.drop(columns='GP', inplace=True, axis=1)
    # delete first names from teams and just keep last name to do the mapping
    nfl_df['team'] = nfl_df['team'].replace({'[\w.]*\ ':''}, regex=True)
    nfl_df['team'] = nfl_df['team'].replace({'\s':''}, regex=True)
    # remove + sign
    nfl_df['team'] = nfl_df['team'].replace({'\+':''}, regex=True)
    # convert dtypes to numeric data
    for i_col in nfl_df:
        if i_col != 'team':
            nfl_df[i_col] = pd.to_numeric(nfl_df[i_col], errors='coerce')

    #rename cities column
    cities.rename(columns= {
        'Population (2016 est.)[8]' : 'Population'
    }, inplace=True)
    # remove brackets from dataset
    cities['NFL'] = cities['NFL'].replace({'\[.*\]':''}, regex=True)
    # mark empty string as nan
    cities.replace('', np.nan, inplace=True)
    # mark "-" as nan ("-" does not work so use sign directly from df)
    cities.replace(cities['NFL'][30], np.nan, inplace=True)
    # convert population series to pd numeric
    cities['Population'] = pd.to_numeric(cities['Population'], errors='coerce')
    #drop columns
    cities.drop(columns=['NHL', 'NBA', 'MLB'], inplace=True)
    # drop nas
    cities.dropna(inplace=True, how='any')
    cities.reset_index(drop=True, inplace=True)
    # split team names by capital and explode to a bigger df and reindex 
    cities = cities.assign(team = [re.findall("(?<!\s)[A-Z](?:[a-z\s]|(?<=\s)[A-Z])*|[0-9]+[a-z]+", i_str) for i_str in cities['NFL']]).explode('team').reset_index(drop=True)
    # drop also first names from team to do the mapping
    cities['team'] = cities['team'].replace({'[\w.]*\ ':''}, regex=True)
    # drop nas
    cities.dropna(inplace=True, how='any')
    cities.reset_index(drop=True, inplace=True)
    # drop old NHL column
    cities.drop(columns=['NFL'], inplace=True)
    # merge dfs
    df4 = pd.merge(cities, nfl_df, 'outer', on='team')
    
    population_by_region = df4.groupby(['Metropolitan area']).mean()['Population'] # pass in metropolitan area population from cities
    win_loss_by_region =  df4.groupby(['Metropolitan area']).mean()['W-L%']

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [8]:
nfl_correlation()

0.004282141436393022

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [9]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

# As a preparation we should change the upper functions so that each function returns the average win-loose rate
# After that we iterate over the different w-l rates and correlate each one with each other except from with itself.
# It is important that you only use the metropolitan regions where there are two sports teams in the current region. 
# So at first the two series have to be merged by inner merging

def nfl_df():
    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    # use only data from 2018
    nfl_df = nfl_df[nfl_df['year']==2018]
    # drop NFC and AFC
    nfl_df = nfl_df.drop(nfl_df[nfl_df['MoV'].str.contains(r'AFC')].index).reset_index(drop=True)
    nfl_df = nfl_df.drop(nfl_df[nfl_df['MoV'].str.contains(r'NFC')].index).reset_index(drop=True)

    # remove division infos
    #nhl_df = nhl_df.drop(nhl_df[nhl_df['GP'].str.contains(r'Division')].index).reset_index(drop=True)
    # remove astericks from data with regexp
    nfl_df['team'] = nfl_df['team'].replace({'\*':''}, regex=True)
    nfl_df['team'] = nfl_df['team'].replace({'\(\d*\)':''}, regex=True)
    # keep only games played, wins, losses and team
    nfl_df = nfl_df[['team', 'W-L%']]
    # drop games played
    #nhl_df.drop(columns='GP', inplace=True, axis=1)
    # delete first names from teams and just keep last name to do the mapping
    nfl_df['team'] = nfl_df['team'].replace({'[\w.]*\ ':''}, regex=True)
    nfl_df['team'] = nfl_df['team'].replace({'\s':''}, regex=True)
    # remove + sign
    nfl_df['team'] = nfl_df['team'].replace({'\+':''}, regex=True)
    # convert dtypes to numeric data
    for i_col in nfl_df:
        if i_col != 'team':
            nfl_df[i_col] = pd.to_numeric(nfl_df[i_col], errors='coerce')

    #rename cities column
    cities.rename(columns= {
        'Population (2016 est.)[8]' : 'Population'
    }, inplace=True)
    # remove brackets from dataset
    cities['NFL'] = cities['NFL'].replace({'\[.*\]':''}, regex=True)
    # mark empty string as nan
    cities.replace('', np.nan, inplace=True)
    # mark "-" as nan ("-" does not work so use sign directly from df)
    cities.replace(cities['NFL'][30], np.nan, inplace=True)
    # convert population series to pd numeric
    cities['Population'] = pd.to_numeric(cities['Population'], errors='coerce')
    #drop columns
    cities.drop(columns=['NHL', 'NBA', 'MLB'], inplace=True)
    # drop nas
    cities.dropna(inplace=True, how='any')
    cities.reset_index(drop=True, inplace=True)
    # split team names by capital and explode to a bigger df and reindex 
    cities = cities.assign(team = [re.findall("(?<!\s)[A-Z](?:[a-z\s]|(?<=\s)[A-Z])*|[0-9]+[a-z]+", i_str) for i_str in cities['NFL']]).explode('team').reset_index(drop=True)
    # drop also first names from team to do the mapping
    cities['team'] = cities['team'].replace({'[\w.]*\ ':''}, regex=True)
    # drop nas
    cities.dropna(inplace=True, how='any')
    cities.reset_index(drop=True, inplace=True)
    # drop old NHL column
    cities.drop(columns=['NFL'], inplace=True)
    # merge dfs
    df4 = pd.merge(cities, nfl_df, 'outer', on='team')
    return df4.groupby(['Metropolitan area']).mean()['W-L%']

def nba_df():
    nba_df=pd.read_csv("assets/nba.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    # use only data from 2018
    nba_df = nba_df[nba_df['year']==2018]

    # remove division infos
    #nhl_df = nhl_df.drop(nhl_df[nhl_df['GP'].str.contains(r'Division')].index).reset_index(drop=True)
    # remove astericks from data with regexp
    nba_df['team'] = nba_df['team'].replace({'\*':''}, regex=True)
    nba_df['team'] = nba_df['team'].replace({'\(\d*\)':''}, regex=True)
    # keep only games played, wins, losses and team
    nba_df = nba_df[['team', 'W/L%']]
    # drop games played
    #nhl_df.drop(columns='GP', inplace=True, axis=1)
    # delete first names from teams and just keep last name to do the mapping
    nba_df['team'] = nba_df['team'].replace({'[\w.]*\ ':''}, regex=True)
    nba_df['team'] = nba_df['team'].replace({'\s':''}, regex=True)
    # convert dtypes to numeric data
    for i_col in nba_df:
        if i_col != 'team':
            nba_df[i_col] = pd.to_numeric(nba_df[i_col], errors='coerce')

    # rename W-L rate to automate merging
    nba_df.rename(columns = {
        'W/L%' : 'W-L%'
    }, inplace=True)
    #rename cities column
    cities.rename(columns= {
        'Population (2016 est.)[8]' : 'Population'
    }, inplace=True)
    # remove brackets from dataset
    cities['NBA'] = cities['NBA'].replace({'\[.*\]':''}, regex=True)
    # mark empty string as nan
    cities.replace('', np.nan, inplace=True)
    # mark "-" as nan ("-" does not work so use sign directly from df)
    cities.replace(cities['NBA'][16], np.nan, inplace=True)
    # convert population series to pd numeric
    cities['Population'] = pd.to_numeric(cities['Population'], errors='coerce')
    #drop columns
    cities.drop(columns=['NHL', 'NFL', 'MLB'], inplace=True)
    # drop nas
    cities.dropna(inplace=True, how='any')
    cities.reset_index(drop=True, inplace=True)
    
    # split team names by capital and explode to a bigger df and reindex 
    cities = cities.assign(team = [re.findall("(?<!\s)[A-Z](?:[a-z\s]|(?<=\s)[A-Z])*|[0-9]+[a-z]+", i_str) for i_str in cities['NBA']]).explode('team').reset_index(drop=True)
    # drop also foírst names from team to do the mapping
    cities['team'] = cities['team'].replace({'[\w.]*\ ':''}, regex=True)  
    # drop old NHL column
    cities.drop(columns=['NBA'], inplace=True)
    # merge dfs
    df2 = pd.merge(cities, nba_df, 'outer', on='team')
    return df2.groupby(['Metropolitan area']).mean()['W-L%']
          
def nhl_df():
    nhl_df=pd.read_csv("assets/nhl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    # cleaning data
    # use only data from 2018
    nhl_df = nhl_df[nhl_df['year']==2018]
    # remove division infos
    nhl_df = nhl_df.drop(nhl_df[nhl_df['GP'].str.contains(r'Division')].index).reset_index(drop=True)
    # remove astericks from data with regexp
    nhl_df['team'] = nhl_df['team'].replace({'\*':''}, regex=True)
    # keep only games played, wins, losses and team
    nhl_df.drop(nhl_df.iloc[:, 4:], inplace = True, axis = 1)
    # drop games played
    nhl_df.drop(columns='GP', inplace=True, axis=1)
    # delete foírst names from teams and just keep last name to do the mapping
    nhl_df['team'] = nhl_df['team'].replace({'[\w.]*\ ':''}, regex=True)
    # convert dtypes to numeric data
    for i_col in nhl_df:
        if i_col != 'team':
            nhl_df[i_col] = pd.to_numeric(nhl_df[i_col], errors='coerce')
    # compute w/l rate
    nhl_df['w_l_rate'] = nhl_df['W'] / (nhl_df['W'] + nhl_df['L'])
    # rename w-l rate to automate it down below
    nhl_df.rename(columns = {
        'w_l_rate' : 'W-L%'
    }, inplace = True)

    # cleaning cities data
    #rename cities column
    cities.rename(columns= {
        'Population (2016 est.)[8]' : 'Population'
    }, inplace=True)
    # remove brackets from dataset
    cities['NHL'] = cities['NHL'].replace({'\[.*\]':''}, regex=True)
    # mark empty string as nan
    cities.replace('', np.nan, inplace=True)
    # mark "-" as nan ("-" does not work so use sign directly from df)
    cities.replace(cities['NHL'][14], np.nan, inplace=True)
    # convert population series to pd numeric
    cities['Population'] = pd.to_numeric(cities['Population'], errors='coerce')
    #drop columns
    cities.drop(columns=['NBA', 'NFL', 'MLB'], inplace=True)
    # drop nas
    cities.dropna(inplace=True, how='any')
    cities.reset_index(drop=True, inplace=True)
    
    # split team names by capital and explode to a bigger df and reindex 
    cities = cities.assign(team = [re.findall("(?<!\s)[A-Z](?:[a-z\s]|(?<=\s)[A-Z])*", i_str) for i_str in cities['NHL']]).explode('team').reset_index(drop=True)
    # drop also foírst names from team to do the mapping
    cities['team'] = cities['team'].replace({'[\w.]*\ ':''}, regex=True)  
    # drop old NHL column
    cities.drop(columns=['NHL'], inplace=True)
    # merge dfs
    df = pd.merge(cities, nhl_df, 'outer', on='team')
    return df.groupby(['Metropolitan area']).mean()['W-L%']


def mlb_df():
    mlb_df=pd.read_csv("assets/mlb.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    # cleaning data
    # use only data from 2018
    mlb_df = mlb_df[mlb_df['year']==2018]
    # remove division infos
    # mlb_df = mlb_df.drop(mlb_df[mlb_df['GP'].str.contains(r'Division')].index).reset_index(drop=True)
    # remove astericks from data with regexp
    mlb_df['team'] = mlb_df['team'].replace({'\*':''}, regex=True)
    # keep only teams and win-loose rate
    mlb_df = mlb_df[['team', 'W-L%']]

    # Because there are red and white concat the names before sox in both dfs to do the mapping
    mlb_df['team'] = mlb_df['team'].str.replace('\ Sox','Sox')
    # delete first names from teams and just keep last name to do the mapping
    mlb_df['team'] = mlb_df['team'].replace({'[\w.]*\ ':''}, regex=True)
    #mlb_df['team'] = mlb_df['team'].replace({'\s':''}, regex=True)
    # convert dtypes to numeric data
    for i_col in mlb_df:
        if i_col != 'team':
            mlb_df[i_col] = pd.to_numeric(mlb_df[i_col], errors='coerce')

    # cleaning cities data
    #rename cities column
    cities.rename(columns= {
        'Population (2016 est.)[8]' : 'Population'
    }, inplace=True)
    # remove brackets from dataset
    cities['MLB'] = cities['MLB'].replace({'\[.*\]':''}, regex=True)
    # mark empty string as nan
    cities.replace('', np.nan, inplace=True)
    # mark "-" as nan ("-" does not work so use sign directly from df)
    cities.replace(cities['MLB'][24], np.nan, inplace=True)
    # convert population series to pd numeric
    cities['Population'] = pd.to_numeric(cities['Population'], errors='coerce')
    #drop columns
    cities.drop(columns=['NBA', 'NFL', 'NHL'], inplace=True)
    # drop nas
    cities.dropna(inplace=True, how='any')
    cities.reset_index(drop=True, inplace=True)
    
    # split team names by capital and explode to a bigger df and reindex 
    cities = cities.assign(team = [re.findall("(?<!\s)[A-Z](?:[a-z\s]|(?<=\s)[A-Z])*", i_str) for i_str in cities['MLB']]).explode('team').reset_index(drop=True)
    # Because there are red and white concat the names before sox in both dfs to do the mapping
    cities['team'] = cities['team'].str.replace('\ Sox','Sox')
    # drop also foírst names from team to do the mapping
    cities['team'] = cities['team'].replace({'[\w.]*\ ':''}, regex=True)  
    # drop old NHL column
    cities.drop(columns=['MLB'], inplace=True)
    # merge dfs
    df3 = pd.merge(cities, mlb_df, 'outer', on='team')
    return df3.groupby(['Metropolitan area']).mean()['W-L%']



def get_df(sport):
    if sport == "NFL":
        return nfl_df()
    elif sport == "NBA":
        return nba_df()
    elif sport == "NHL":
        return nhl_df()
    elif sport == "MLB":
        return mlb_df()

def sports_team_performance():
    # YOUR CODE HERE

    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    for i_sport in sports:
        for j_sport in sports:
            if i_sport != j_sport:
                merged_df = pd.merge(get_df(i_sport), get_df(j_sport), how="inner", on=['Metropolitan area'])
                # returns two values (statistics and pvalue) --> return second value
                p_values.loc[i_sport, j_sport]=stats.ttest_rel(merged_df['W-L%_x'],merged_df['W-L%_y'])[1]
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values

In [10]:
sports_team_performance()

<ipython-input-9-d11aeb885589>:206: FutureWarning: The default value of regex will change from True to False in a future version.
  mlb_df['team'] = mlb_df['team'].str.replace('\ Sox','Sox')
<ipython-input-9-d11aeb885589>:237: FutureWarning: The default value of regex will change from True to False in a future version.
  cities['team'] = cities['team'].str.replace('\ Sox','Sox')
<ipython-input-9-d11aeb885589>:206: FutureWarning: The default value of regex will change from True to False in a future version.
  mlb_df['team'] = mlb_df['team'].str.replace('\ Sox','Sox')
<ipython-input-9-d11aeb885589>:237: FutureWarning: The default value of regex will change from True to False in a future version.
  cities['team'] = cities['team'].str.replace('\ Sox','Sox')
<ipython-input-9-d11aeb885589>:206: FutureWarning: The default value of regex will change from True to False in a future version.
  mlb_df['team'] = mlb_df['team'].str.replace('\ Sox','Sox')
<ipython-input-9-d11aeb885589>:237: FutureWar

,NFL,NBA,NHL,MLB
NFL,NaN,0.937509,0.030318,0.803459
NBA,0.937509,NaN,0.022386,0.949566
NHL,0.030318,0.022386,NaN,0.000703
MLB,0.803459,0.949566,0.000703,NaN


In [11]:
merge=pd.merge(get_df('NHL'),get_df('MLB'),'inner', on = ['Metropolitan area'])

<ipython-input-9-d11aeb885589>:206: FutureWarning: The default value of regex will change from True to False in a future version.
  mlb_df['team'] = mlb_df['team'].str.replace('\ Sox','Sox')
<ipython-input-9-d11aeb885589>:237: FutureWarning: The default value of regex will change from True to False in a future version.
  cities['team'] = cities['team'].str.replace('\ Sox','Sox')


In [12]:
merge

,W-L%_x,W-L%_y
Metropolitan area,,
Boston,0.714286,0.667
Chicago,0.458333,0.483
Dallas–Fort Worth,0.567568,0.414
Denver,0.589041,0.558
Detroit,0.434783,0.395
Los Angeles,0.622895,0.529
Miami–Fort Lauderdale,0.594595,0.391
Minneapolis–Saint Paul,0.633803,0.481
New York City,0.518201,0.546


In [13]:
stats.ttest_rel(merge['W-L%_x'],merge['W-L%_y'])

Ttest_relResult(statistic=4.183064406351442, pvalue=0.0007029593210460182)